ADP data by Steven Cunningham 06/16/17

This code is intened for take SonTek ADP data
- First determine seafloor by signal amplitude file ".a#". The signal should slowly decay through water, when decay is
  interupted with amplitude spike, then the signal is bouncing back from the floor. 
- Compare one bin to the previous until the next bin is larger than previous.
- Number of good profiles will be used on velocity data. 

In [57]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import csv


In [58]:
path = 'C:\\Users\\steve\\Desktop\\Thesis\\ADP files\\10.13.16'
file_group = 'SWC0610131152'
sig_amp = '%s.a2'% file_group
VE_file = '%s.ve'% file_group
VN_file = '%s.vn'% file_group
VU_file = '%s.vu'% file_group
save_name=(sig_amp[:-3]+'done') # save file name string
save_file_name = (path[-8:])
df = pd.read_csv(sig_amp, delimiter='\s+', header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,197,106,106,93,83,73,67,63,61,...,46,67,131,105,51,33,27,25,24,24
1,2,158,132,107,100,93,85,83,76,73,...,46,72,146,107,51,36,30,27,26,25
2,3,150,123,106,99,91,86,84,81,75,...,45,76,145,98,49,33,28,26,25,25
3,4,155,131,105,91,82,73,66,61,56,...,42,70,139,95,46,34,29,27,25,24
4,5,142,129,103,88,78,73,71,73,78,...,44,76,126,90,45,34,29,27,25,24


In [59]:
mean = np.mean(df, axis = 0)[1:] # average profiles across a depth bin
mean2 = pd.Series.to_frame(mean, 'average_profile') # transform to dataframe 
mean2["next_profile"] = mean2["average_profile"].shift(-1) # Shift down to commpare difference 
mean2["Nartural_decay"] = mean2["average_profile"] > mean2["next_profile"] # check if the next bin is greater than previous 
mean2["True_profile"] = mean2["Nartural_decay"].cumsum() # gives values to groups of true/false
mean2["Number"] = np.arange(23)[1:] # make a range of values 
mean2["good_profile"] = mean2["True_profile"] == mean2["Number"] # see if values assigned to true/false match number order  
print(mean2)

how_many=sum(mean2["good_profile"]) # how many good profiles are there. This will be used on velocity data 
mean3=mean2["average_profile"][:how_many+1] # get profile values for good bins. Add 1 because the .shift 


    average_profile  next_profile Nartural_decay  True_profile  Number  \
1           160.375       125.750           True             1       1   
2           125.750       102.750           True             2       2   
3           102.750        92.500           True             3       3   
4            92.500        83.375           True             4       4   
5            83.375        76.375           True             5       5   
6            76.375        73.500           True             6       6   
7            73.500        73.250           True             7       7   
8            73.250        72.875           True             8       8   
9            72.875        64.750           True             9       9   
10           64.750        53.000           True            10      10   
11           53.000        43.875           True            11      11   
12           43.875        45.500          False            11      12   
13           45.500        75.500     

Now calculate current speed by averaging velocities and direction 

In [60]:


VE = pd.read_csv(VE_file, delimiter='\s+', header=None)
VE.head() #depth = axis 0 = profile, axis 1 = depth
VN = pd.read_csv(VN_file, delimiter='\s+', header=None)
VU = pd.read_csv(VU_file, delimiter='\s+', header=None)
VU.head() #depth = axis 0 = profile, axis 1 = depth

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,0.3,0.6,0.8,1.1,0.9,0.9,1.2,0.3,1.1,...,1.1,2.6,2.5,2.8,8.7,12.1,25.1,29.8,41.2,49.7
1,2,1.4,0.7,0.2,0.4,0.0,0.0,0.0,0.7,0.8,...,1.0,1.6,0.4,0.3,4.1,8.6,13.2,22.9,46.8,62.7
2,3,1.5,1.3,0.7,0.7,0.4,0.2,0.5,0.6,-0.6,...,1.6,2.6,1.1,1.1,2.6,6.0,12.5,28.5,50.6,63.0
3,4,1.9,1.3,0.7,0.7,0.0,0.8,0.6,0.6,0.3,...,1.5,1.0,0.7,0.7,8.1,16.3,20.2,33.6,57.1,72.8
4,5,0.2,0.1,1.0,0.4,0.3,0.0,-0.3,0.6,-0.1,...,2.6,1.6,-0.2,2.6,9.9,19.4,28.1,38.2,62.3,78.4


In [61]:
VE=VE.iloc[:,1:how_many +1] # use how_many from aplitude to index good velocity profiles, add one because of 0 index
VE.head()
VN=VN.iloc[:,1:how_many +1] # use how_many from aplitude to index good velocity profiles, add one because of 0 index
VU = VU.iloc[:,1:how_many +1]
VU.head()

,1,2,3,4,5,6,7,8,9,10,11
0,0.3,0.6,0.8,1.1,0.9,0.9,1.2,0.3,1.1,0.2,1.7
1,1.4,0.7,0.2,0.4,0.0,0.0,0.0,0.7,0.8,0.0,0.5
2,1.5,1.3,0.7,0.7,0.4,0.2,0.5,0.6,-0.6,0.0,0.0
3,1.9,1.3,0.7,0.7,0.0,0.8,0.6,0.6,0.3,0.3,1.8
4,0.2,0.1,1.0,0.4,0.3,0.0,-0.3,0.6,-0.1,0.7,0.6


In [62]:
# average across same depth and round to .001
VEV= np.array(np.round(np.mean(VE, axis = 0), 3)) 
VNV = np.array(np.round(np.mean(VN, axis = 0), 3))
VUV = np.array(np.round(np.mean(VU, axis = 0), 3))
NE_vector = (VEV**2 + VNV**2)**.5
print(NE_vector)
print(VNV, VEV)
print(VUV)

[ 3.10217875  1.97119887  1.36190602  1.46756397  1.27466427  1.56414322
  2.44502045  1.20929236  1.67040235  2.11261094  2.46234137]
[-2.587 -0.525 -1.262 -1.412 -0.775 -1.112 -1.95  -0.662 -1.    -1.35  -1.85 ] [ 1.712  1.9    0.512  0.4    1.012  1.1    1.475  1.012  1.338  1.625
  1.625]
[ 1.35   0.675  0.612  0.538  0.25   0.125  0.2    0.238  0.113  0.088
  0.487]


In [63]:
North_sign = VNV > 0
East_sign = VEV >0

In [64]:
VNV = np.array(VNV) # change from DataFrame to np.array
VEV = np.array(VEV)
resultsN = []
for i in (VNV):    #make lists of the direction from positive or negative velocities 
    if i > 0:
        resultsN.extend("N")
    else:
        resultsN.extend("S")
print(resultsN)

['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S']


In [65]:
resultsE = []
for i in (VEV):
    if i > 0:
        resultsE.extend("E")
    else:
        resultsE.extend("W")
print(resultsE)

['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']


In [66]:

depth = np.arange(1,how_many+1)
depth = depth*.7

total_velocity = pd.DataFrame(NE_vector, columns = ["Velocity [cm/s]"])
total_velocity['ADP_file'] = sig_amp[:-3]
total_velocity['Depth (m)'] = depth
total_velocity['Direction1'] = resultsN
total_velocity['Direction2'] = resultsE
total_velocity['verticle_velocity [cm/s]'] = VUV
total_velocity = total_velocity[['ADP_file', 'Velocity [cm/s]', 'verticle_velocity [cm/s]', 'Depth (m)', 'Direction1', 'Direction2']] 

print(total_velocity)
print(save_name)

         ADP_file  Velocity [cm/s]  verticle_velocity [cm/s]  Depth (m)  \
0   SWC0610131152         3.102179                     1.350        0.7   
1   SWC0610131152         1.971199                     0.675        1.4   
2   SWC0610131152         1.361906                     0.612        2.1   
3   SWC0610131152         1.467564                     0.538        2.8   
4   SWC0610131152         1.274664                     0.250        3.5   
5   SWC0610131152         1.564143                     0.125        4.2   
6   SWC0610131152         2.445020                     0.200        4.9   
7   SWC0610131152         1.209292                     0.238        5.6   
8   SWC0610131152         1.670402                     0.113        6.3   
9   SWC0610131152         2.112611                     0.088        7.0   
10  SWC0610131152         2.462341                     0.487        7.7   

   Direction1 Direction2  
0           S          E  
1           S          E  
2           S     

In [67]:
filename = "%s.csv" % save_name

total_velocity.to_csv(filename)